In [20]:
# Dependencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import zipfile
from pycocotools.coco import COCO
import os
import shutil
from PIL import Image
from tqdm.notebook import tqdm 
import json
import import_ipynb
import nbformat
import cv2

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard


# Created models/functions for pre-processing data
from pre_processing import download_coco_datasets as dcd
from pre_processing import filter_coco_datasets as fcd
from pre_processing import resize_coco_datasets as rcd
from pre_processing import resize_annotations as ra

In [17]:
# Function to normalize pixel values to range [0,1]
def normalize_min_max(image):
    return image / 255.0

In [18]:
# Loading images as an numpy-array from directory
def load_images_from_directory(directory):
    images = []
    file_list = [f for f in os.listdir(directory) if f.endswith('.jpg') or f.endswith('.jpeg')]
    for filename in tqdm(file_list, desc="Loading Images"):
        img = cv2.imread(os.path.join(directory, filename))
        # Convert to RGB format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Normalize pixel values
        img = normalize_min_max(img)
        images.append(img)
    return np.array(images)

In [19]:
def load_annotations_from_json(json_file):
    with open(json_file, 'r') as f:
        annotations = json.load(f)
    # Parse annotations as needed and return
    return tqdm(annotations, desc="Loading Annotations")

In [12]:
# Path to pre-processed data directory
pre_processed_data = './coco_datasets/resized_images/'

# loading images as a numpy-array
train_data = load_images_from_directory(os.path.join(pre_processed_data, 'train2017_resized'))
test_data = load_images_from_directory(os.path.join(pre_processed_data, 'test2017_resized'))
val_data = load_images_from_directory(os.path.join(pre_processed_data, 'val2017_resized'))


# Loading size adjusted annotations from directory
annotations_dir = './coco_datasets/adjusted_annotations/
train_annotations = load_annotations_from_json(os.path.join(annotations_dir, 'instances_train2017.json'))
val_annotations = load_annotations_from_json(os.path.join(annotations_dir, 'instances_val2017.json'))

Loading Images:   0%|          | 0/118287 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 244x244 pixels, 3 channels (RGB)
input_shape = (224, 224, 3)
num_classes = 5  # xmin, ymin, xmax, ymax, class

# Initialize the model
model = Sequential()

# Add layers to the model
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes))  # Output layer

In [ ]:
# Compiling the model, using adam as optimizer and mean square error as loss function
model.compile(optimizer='adam', loss='mse') 

In [ ]:
# Creating a log directory for visualization logs
# tensorboard --logdir logs/fit    # use in command to get url link for logs

log_dir = "logs/fit/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# training the model with 10 epochs and a batch size of 32
model.fit(train_data, train_annotations, validation_data=(val_data, val_annotations), epochs=10, batch_size=32, callbacks=[tensorboard_callback])

In [ ]:
# Evaluating the model on the test data
model.evaluate(test_data)

In [ ]:
# Saving the model
model.save('car_detector_model.h5')

In [ ]:
loaded_model = load_model('car_detector_model.h5')
predictions = loaded_model.predict(new_images) # REMOVE new_images for other data